In [1]:
# imports
import pandas as pd
import numpy as np
from google.cloud import storage
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import joblib
from sklearn.metrics import accuracy_score
import pickle

In [2]:
# create storage client to download data
storage_client = storage.Client()
public_bucket = storage_client.bucket('sba_data_uoftcloudrj')
blob = public_bucket.blob('training_data.csv')
blob.download_to_filename('training_data.csv')

In [3]:
df = pd.read_csv('training_data.csv')

In [4]:
df.head()

,LoanNr_ChkDgt,Zip,NAICS,ApprovalFY,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,MIS_Status,SBA_Appv
0,2216105006,39530,45,2007,84,2,1.0,2,4,0,1,0,0,25000.0,0,21250.0
1,3101536007,33176,62,2008,84,10,1.0,1,10,1,1,1,0,51605.0,0,15000.0
2,1815165010,97045,45,2006,84,2,2.0,2,4,0,1,1,0,32000.0,0,16000.0
3,3078164001,11764,81,1999,48,3,1.0,3,3,1,1,1,0,99769.0,0,25000.0
4,3320723006,90019,81,1989,221,6,1.0,0,0,0,0,0,0,550000.0,0,467500.0


In [5]:
df.set_index('LoanNr_ChkDgt', inplace=True)

In [6]:
X = df[['Zip','NAICS','ApprovalFY','Term','NoEmp','NewExist','CreateJob','RetainedJob','FranchiseCode','UrbanRural','RevLineCr','LowDoc','DisbursementGross','SBA_Appv']]
y = df['MIS_Status']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [8]:
rf = RandomForestClassifier(random_state=42, max_depth=10)
rf.fit(X_train, y_train)
pred = rf.predict(X_test)

In [9]:
accuracy_score(y_test, pred)

0.9212106537530267

In [11]:
pickle.dump(rf, open('model.pkl','wb'))

In [12]:
storage_client = storage.Client()
bucket = storage_client.bucket('sba_data_uoftcloudrj')
blob = bucket.blob('model.pkl')
blob.upload_from_filename('model.pkl')